In [ ]:
!nvidia-smi # to see what GPU you have

Mon Apr 24 20:01:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


### Levenshtein

This may take a while

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.7 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 24.45 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14147

## Imports

In [ ]:
import torch
import random
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime
import multiprocessing


# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"shekennotstirred","key":"e5294822ccdd2e7e243b3b9a9efbef80"}') 

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73272 sha256=d1e7cdebcfa064f908b67a2876c1a7c605d40bd8bcf39770cf8bf479c98f846e
  Stored in directory: /root/.cache/pip/wheels/d4/02/ef/3f8c8d86b8d5388a1d3155876837f1a1a3143ab3fc2ff1ffad
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.13
    Uninstalling kaggle-1.5.13:
      Successfully uninstalled kaggle-1.5.13


In [ ]:
!kaggle competitions download -c 11-785-s23-hw3p2

100% 15.9G/15.9G [01:09<00:00, 301MB/s]
100% 15.9G/15.9G [01:09<00:00, 248MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
!mkdir '/content/data'

!unzip -qo '11-785-s23-hw3p2.zip' -d '/content/data'
!ls

11-785-s23-hw3p2.zip  ctcdecode  data  sample_data


In [ ]:
!mkdir '/content/data/11-785-s23-hw3p2/train-clean-460'
!mkdir '/content/data/11-785-s23-hw3p2/train-clean-460/mfcc'

!zip -r -j -qo /content/data/11-785-s23-hw3p2/train-clean-460/mfcc/100mfcc.zip /content/data/11-785-s23-hw3p2/train-clean-100/mfcc
!zip -r -j -qo /content/data/11-785-s23-hw3p2/train-clean-460/mfcc/360mfcc.zip /content/data/11-785-s23-hw3p2/train-clean-360/mfcc

!unzip -qo '/content/data/11-785-s23-hw3p2/train-clean-460/mfcc/100mfcc.zip' -d '/content/data/11-785-s23-hw3p2/train-clean-460/mfcc'
!unzip -qo '/content/data/11-785-s23-hw3p2/train-clean-460/mfcc/360mfcc.zip' -d '/content/data/11-785-s23-hw3p2/train-clean-460/mfcc'

!rm '/content/data/11-785-s23-hw3p2/train-clean-460/mfcc/100mfcc.zip'
!rm '/content/data/11-785-s23-hw3p2/train-clean-460/mfcc/360mfcc.zip'

In [ ]:
!mkdir '/content/data/11-785-s23-hw3p2/train-clean-460/transcript'

!zip -r -j -qo /content/data/11-785-s23-hw3p2/train-clean-460/transcript/100trans.zip /content/data/11-785-s23-hw3p2/train-clean-100/transcript
!zip -r -j -qo /content/data/11-785-s23-hw3p2/train-clean-460/transcript/360trans.zip /content/data/11-785-s23-hw3p2/train-clean-360/transcript

!unzip -qo '/content/data/11-785-s23-hw3p2/train-clean-460/transcript/100trans.zip' -d '/content/data/11-785-s23-hw3p2/train-clean-460/transcript'
!unzip -qo '/content/data/11-785-s23-hw3p2/train-clean-460/transcript/360trans.zip' -d '/content/data/11-785-s23-hw3p2/train-clean-460/transcript'

!rm '/content/data/11-785-s23-hw3p2/train-clean-460/transcript/100trans.zip'
!rm '/content/data/11-785-s23-hw3p2/train-clean-460/transcript/360trans.zip'

# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
# You might want to play around with the mapping as a sanity check here

### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, data_directory): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = data_directory + '/mfcc/'
        self.transcript_dir = data_directory + '/transcript/'

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(self.mfcc_files) == len(self.transcript_files)

        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        self.mfccs = []
        self.transcripts = []
        # Iterate through mfccs and transcripts
        for i in range(self.length):
          mfcc = np.load(self.mfcc_dir + self.mfcc_files[i])
          # Do Cepstral Normalization of mfcc (explained in writeup)
          mfcc = (mfcc - np.mean(mfcc, axis=0, keepdims=True)) / np.std(mfcc, axis=0, keepdims=True)
          # Load the corresponding transcript
          # Remove [SOS] and [EOS] from the transcript 
          # Note that SOS will always be in the starting and EOS at end, as the name suggests.
          transcript  = np.load(self.transcript_dir + self.transcript_files[i])[1:-1]
          # Map the String phoneme value to integer values for our model
          transcript = np.array([self.PHONEMES.index(phoneme) for phoneme in transcript])
          self.mfccs.append(mfcc)
          self.transcripts.append(transcript)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        '''
        RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        # Convert the MFCC array to a tensor
        mfcc = torch.tensor(self.mfccs[ind])

        # Convert the transcript to a tensor of phoneme indices
        transcript = torch.tensor(self.transcripts[ind])

        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        # batch of output phonemes
        batch_mfcc, batch_transcript = zip(*batch)
        
        # pad mfcc and transcript to have the same length i.e. the length of the
        # longest sequence in the given mini batch
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True)

        # store actual (unpadded) length of each recording and transcript
        lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc]
        lengths_transcript = [len(transcript) for transcript in batch_transcript]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
    def __init__(self, data_directory): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = data_directory + '/mfcc/'

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

        self.PHONEMES = PHONEMES

        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        self.mfccs = []
        # Iterate through mfccs and transcripts
        for i in range(self.length):
          mfcc = np.load(self.mfcc_dir + self.mfcc_files[i])
          # Do Cepstral Normalization of mfcc (explained in writeup)
          mfcc = (mfcc - np.mean(mfcc, axis=0, keepdims=True)) / np.std(mfcc, axis=0, keepdims=True)
          self.mfccs.append(mfcc)
          
    def __len__(self): 
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        # Convert the MFCC array to a tensor
        mfcc = torch.tensor(self.mfccs[ind])
        return mfcc

    def collate_fn(self,batch):
        '''
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        # batch of output phonemes
        batch_mfcc = batch
        
        # pad mfcc to have the same length i.e. the length of the
        # longest sequence in the given mini batch
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)

        # store actual (unpadded) length of each recording
        lengths_mfcc = [len(mfcc) for mfcc in batch_mfcc]

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

### Data - Hyperparameters

In [ ]:
config = {
    "beam_width" : 2,
    "lr" : 1e-3,
    'batch_size' : 64,
    "epochs" : 50,
    "patience": 2, 
    "factor": 0.5,
    "num_layers": 2,
    "dropout": 0.2,
    } # Feel free to add more items here

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation

root = '/content/hw3p2'

### Data loaders

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

0

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(data_directory='/content/data/11-785-s23-hw3p2/train-clean-360/')
val_data = AudioDataset(data_directory='/content/data/11-785-s23-hw3p2/dev-clean/')
test_data = AudioDatasetTest(data_directory='/content/data/11-785-s23-hw3p2/test-clean/')

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data, 
    num_workers = 4,
    batch_size  = config['batch_size'], 
    pin_memory  = True,
    shuffle     = True,
    collate_fn  = train_data.collate_fn
)
val_loader = torch.utils.data.DataLoader(
    dataset     = val_data, 
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
    collate_fn  = val_data.collate_fn
)
test_loader = torch.utils.data.DataLoader(
    dataset     = test_data, 
    num_workers = 2, 
    batch_size  = config['batch_size'], 
    pin_memory  = True, 
    shuffle     = False,
    collate_fn  = test_data.collate_fn
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 104013, batches = 1626
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 
for data in val_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break 
for data in test_loader:
    x, lx = data
    print(x.shape, lx.shape)
    break 

torch.Size([64, 1625, 27]) torch.Size([64, 202]) torch.Size([64]) torch.Size([64])
torch.Size([64, 2936, 27]) torch.Size([64, 265]) torch.Size([64]) torch.Size([64])
torch.Size([64, 2001, 27]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
# torch.cuda.empty_cache()

# class Network(nn.Module):

#     def __init__(self, input_size, output_size):
#         super(Network, self).__init__()

#         # Adding some sort of embedding layer or feature extractor might help performance.
#         # self.embedding = ?
        
#         # TODO : look up the documentation. You might need to pass some additional parameters.
#         self.lstm = nn.LSTM(input_size=input_size, hidden_size=256, num_layers=1, bidirectional=True)
       
#         self.classification = nn.Sequential(
#             nn.Linear(512, output_size)
#             #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
#         )

#         self.logSoftmax = nn.LogSoftmax(dim=2) #TODO: Apply a log softmax here. Which dimension would apply it on ?

#     def forward(self, x, lx):
#         #TODO
#         # The forward function takes 2 parameter inputs here. Why?
#         # Refer to the handout for hints
#         # x has shape (batch_size, max_seq_len, input_size)
#         # lx is a tensor of sequence lengths, of shape (batch_size,)
        
#         x = x.permute(1, 0, 2)  # (max_seq_len, batch_size, input_size)
#         packed_x = nn.utils.rnn.pack_padded_sequence(x, lx, enforce_sorted=False)
#         packed_h, _ = self.lstm(packed_x)
#         h, lh = nn.utils.rnn.pad_packed_sequence(packed_h)
#         # h has shape (max_seq_len, batch_size, 512)

#         # Apply linear layer and logsoftmax
#         h = self.classification(h)
#         h = self.logSoftmax(h)
#         # h has shape (max_seq_len, batch_size, output_size)

#         return h.permute(1, 0, 2), lh

## Pyramid Bi-LSTM (pBLSTM)

In [ ]:
torch.cuda.empty_cache()


In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    '''
    PyTorch conv1d expects tensors of shape (N, C, L)
    Permuting the input aligns the feature dim with C
    (B, T, 27) -> (B, 27, T)
    '''
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''
    
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        # Initialize a single layer bidirectional LSTM with the given input_size and hidden_size
        self.blstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=config['num_layers'], batch_first=True, bidirectional=True)

    def forward(self, x_packed): # x_packed is a PackedSequence

        # Pad Packed Sequence 
        if isinstance(x_packed, torch.nn.utils.rnn.PackedSequence):
          # pad a packed sequence after computation from the RNN
          # seq_unpacked, lens_unpacked = pad_packed_sequence(packed, batch first=True)
          x, x_lens = pad_packed_sequence(x_packed, batch_first=True)
        else:
          x = x_packed
          x_lens = None

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        x, x_lens = self.trunc_reshape(x, x_lens)
        
        # Pack Padded Sequence. What output(s) would you get?
        if x_lens is not None:
          # pack a padded sequence before passing it through an RNN
          # packed = pack_padded_sequence(recordings, length_of_each_recording, batch_first=True, enforce_sorted=False)
          x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        else:
          x_packed = x

        # Pass the sequence through bLSTM
        output, (h_n, c_n) = self.blstm(x_packed)

        # What do you return?
        return output

    def trunc_reshape(self, x, x_lens): 
        batch_size, length, features = x.shape
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        '''
        If the length of the original input sequence is odd, then it is either padded out by appending
        an extra vector of zeros at the end, before reshaping, or trimmed to an even length by deleting
        the final vector. 
        '''
        if (length % 2) != 0:
          # trim to an even length by deleting the final vector and decrementing length
          x = x[:, :-1, :]
          x_lens[-1] -= 1
        
        downsampled_length = length // 2
        # Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        x = torch.reshape(x, shape=(batch_size, downsampled_length, 2 * features))
        # Reduce lengths by the same downsampling factor = 2
        x_lens = torch.clamp(x_lens, max=downsampled_length, out=None)
        return x, x_lens

# Encoder

In [ ]:
class LockedDropout(nn.Module):
  '''
  Locked dropout is a variation of dropout that is specifically designed for use in 
  the parallel bidirectional LSTM (pBLSTM) architecture. It uses the same dropout mask 
  for both the forward and backward layers.  This ensures that the same set of neurons 
  are dropped out at each time step for both layers, maintaining consistency across time.

  From https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py
  '''
  def __init__(self):
    super().__init__()

  def forward(self, x, dropout=config['dropout']):
    if not self.training or not dropout:
      return x
    x, x_lens = pad_packed_sequence(x, batch_first=True)
    m = x.new(x.shape[0],1,x.shape[2]).bernoulli_(1-dropout)
    mask = Variable(m, requires_grad=False) / (1 - dropout)
    mask = mask.expand_as(x)
    x = mask * x
    # pack a padded sequence using before passing it through an RNN
    x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
    return x_packed

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, embed_size, hidden_size):
        super(Encoder, self).__init__()

        # You can use CNNs as Embedding layer to extract features. 
        # Keep in mind the Input dimensions and expected dimension of Pytorch CNN.
        self.embedding = torch.nn.Sequential(
            PermuteBlock(), # transpose to (N, C, L) for Conv1d
            nn.Conv1d(input_size, embed_size, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(embed_size),
            PermuteBlock(), # transpose back for RNN
        )
        # How many pBLSTMs are required?
        # Fill this up with pBLSTMs - What should the input_size be? 
        # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
        self.pBLSTMs = torch.nn.Sequential(
            pBLSTM(input_size=2*embed_size, hidden_size=hidden_size),
            LockedDropout(),
            pBLSTM(input_size=4*hidden_size, hidden_size=hidden_size),
            LockedDropout(),
        )          
    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        
        # Call the embedding layer
        x = self.embedding(x)
        
        # Pack padded sequence
        x = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)

        # Pass Sequence through the pyramidal Bi-LSTM layer
        x = self.pBLSTMs(x)

        # Pad packed sequence
        x, x_lens = pad_packed_sequence(x, batch_first=True)

        # Return encoder outputs and encoder sequence lengths
        return x, x_lens

# Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(2 * embed_size), PermuteBlock(),
            #Use Permute Block before and after BatchNorm1d() to match the size
            torch.nn.Linear(2*embed_size, 2048),
            PermuteBlock(), torch.nn.BatchNorm1d(2048), PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Linear(2048, 2048),
            PermuteBlock(), torch.nn.BatchNorm1d(2048), PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(2048, output_size)
        )
        
        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        # call your MLP
        out = self.mlp(encoder_out)
        # Think what should be the final output of the decoder for the classification 
        out = self.softmax(out)

        return out 

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size, hidden_size, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations = torch.nn.Sequential(
            # Add Time Masking/ Frequency Masking
            # Hint: See how to use PermuteBlock() function defined above
            PermuteBlock(),
            tat.FrequencyMasking(freq_mask_param=10),
            tat.TimeMasking(time_mask_param=40),
            PermuteBlock(),
        )
        # Initialize Encoder
        self.encoder = Encoder(input_size=input_size, embed_size = embed_size, hidden_size=hidden_size)
        # Initialize Decoder
        self.decoder = Decoder(embed_size=hidden_size, output_size=output_size) 
        
    
    def forward(self, x, lengths_x):
        if self.training:
            x = self.augmentations(x)
        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## INIT
(If trying out the basic Network)

In [ ]:
# torch.cuda.empty_cache()

# model = Network(
#     input_size  = 27,
#     output_size = len(PHONEMES)).to(device)
# summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

## INIT ASR

In [ ]:
model = ASRModel(
    input_size  = 27,
    embed_size  = 256,
    hidden_size = 256,
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): FrequencyMasking()
    (2): TimeMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Conv1d(27, 256, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): PermuteBlock()
    )
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(512, 256, num_layers=2, batch_first=True, bidirectional=True)
      )
      (1): LockedDropout()
      (2): pBLSTM(
        (blstm): LSTM(1024, 256, num_layers=2, batch_first=True, bidirectional=True)
      )
      (3): LockedDropout()
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Linear(in_features=512, out_features=2048, bias=True)
   

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations.PermuteBlock_0,-,"[128, 27, 2363]",NaN,NaN
1_augmentations.FrequencyMasking_1,-,"[128, 27, 2363]",NaN,NaN
2_augmentations.TimeMasking_2,-,"[128, 27, 2363]",NaN,NaN
3_augmentations.PermuteBlock_3,-,"[128, 2363, 27]",NaN,NaN
4_encoder.embedding.PermuteBlock_0,-,"[128, 27, 2363]",NaN,NaN
5_encoder.embedding.Conv1d_1,"[27, 256, 3]","[128, 256, 2363]",20736.0,48999168.0
6_encoder.embedding.BatchNorm1d_2,[256],"[128, 256, 2363]",512.0,256.0
7_encoder.embedding.PermuteBlock_3,-,"[128, 2363, 256]",NaN,NaN
8_encoder.pBLSTMs.0.LSTM_blstm,-,"[94678, 512]",3153920.0,3145728.0


# Training Config

In [ ]:
# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
criterion = torch.nn.CTCLoss(blank=0, reduction='mean') 

optimizer =  torch.optim.AdamW(model.parameters(), lr=config['lr'])

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(
    LABELS,
    alpha=0,
    beta=0,
    cutoff_top_n=len(LABELS),
    cutoff_prob=1.0,
    beam_width=config['beam_width'],
    num_processes=multiprocessing.cpu_count(),
    blank_id=0,
    log_probs_input=True
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=config['patience'], factor=config['factor'])

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

## Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):
    
    # look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_lens = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings                    = []
    
    for i in range(output_lens.shape[0]):
        # Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        prediction = ""
        for j in range(out_lens[i, 0]):
            prediction += PHONEME_MAP[beam_results[i, 0, j]]
        pred_strings.append(prediction)
    
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers
    
    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    label_strings = ["".join(PHONEME_MAP[i] for i in label[j, :label_lens[j]].tolist()) for j in range(batch_size)]
    for i in range(batch_size):
        dist += Levenshtein.distance(pred_strings[i], label_strings[i])
    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0): #TODO: valid_loader
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS)) #TODO: valid_decoder
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    
    break

torch.Size([128, 734, 41])
167.5390625
torch.Size([734, 128, 41]) torch.Size([128, 265])
tensor(30.4252, device='cuda:0', grad_fn=<MeanBackward0>)


## wandb

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="a948a7a6118c9cb14a779f2fbfbf9f4ea1b82754")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "HW3P2-gcp-360", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account 
    config = config ### Wandb Config for your run
)

In [ ]:
# run.finish()

# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():     
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [ ]:
!nvidia-smi

Mon Apr 24 21:55:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    30W /  70W |   8805MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    
    break

torch.Size([64, 734, 41])
170.84375
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(32.0557, device='cuda:0', grad_fn=<MeanBackward0>)


### Training Setup

In [ ]:
def save_model(model, file, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1], 
         'epoch'                    : epoch}, 
         path + file +'.pth'
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# model, optimizer, scheduler, epoch, metric = load_model('/content/best_models/hw3p2_460_model_checkpoint.pth', model, 'valid_dist')

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/content/epoch_models" #TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "/content/best_models" #TODO set best model path 

In [ ]:
torch.cuda.empty_cache()
gc.collect()

# complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))
    
    curr_lr = float(optimizer.param_groups[0]['lr'])#TODO

    train_loss              = train_model(model, train_loader, criterion, optimizer) #TODO
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS) #TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,  
        'valid_dist': valid_dist, 
        'valid_loss': valid_loss, 
        'lr'        : curr_lr
    })
    
    save_model(model, '/hw3p2_460_2_model_checkpoint_'+str(epoch), optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, '/hw3p2_460_2_model_checkpoint', optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/50


	Train Loss 1.1546	 Learning Rate 0.0010000
	Val Dist 9.9003%	 Val Loss 0.4623


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saved epoch model
Saved best model

Epoch: 2/50


	Train Loss 0.5228	 Learning Rate 0.0010000
	Val Dist 7.1981%	 Val Loss 0.3311
Saved epoch model
Saved best model

Epoch: 3/50


...

	Train Loss 0.0916	 Learning Rate 0.0005000
	Val Dist 2.7499%	 Val Loss 0.1563
Saved epoch model

Epoch: 47/50


	Train Loss 0.0891	 Learning Rate 0.0005000
	Val Dist 2.7275%	 Val Loss 0.1595
Saved epoch model

Epoch: 48/50


	Train Loss 0.0797	 Learning Rate 0.0002500
	Val Dist 2.6134%	 Val Loss 0.1574
Saved epoch model
Saved best model

Epoch: 49/50


	Train Loss 0.0792	 Learning Rate 0.0002500
	Val Dist 2.6143%	 Val Loss 0.1548
Saved epoch model

Epoch: 50/50


	Train Loss 0.0767	 Learning Rate 0.0002500
	Val Dist 2.6359%	 Val Loss 0.1591
Saved epoch model


In [58]:
run.finish()

lr,████████████████████████████████▃▃▃▃▃▃▁▁
train_loss,█▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_dist,█▅▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▅▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,0.00025
train_loss,0.07673
valid_dist,2.63588
valid_loss,0.15906


# Generate Predictions and Submit to Kaggle

In [59]:
# Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 10

# test_decoder    = CTCBeamDecoder(labels=PHONEMES, beam_width=TEST_BEAM_WIDTH) #TODO
test_decoder = CTCBeamDecoder(
    PHONEMES,
    beam_width=TEST_BEAM_WIDTH,
    blank_id=0,
    log_probs_input=True
)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string= decode_prediction(h, lh, test_decoder) # TODO call decode_prediction 
    # save the output in results array.
    results.append(prediction_string)
    
    del x, lx, h, lh
    torch.cuda.empty_cache()

results = [element for i in results for element in i]

Testing


100%|██████████| 41/41 [00:36<00:00,  1.12it/s]


In [ ]:
data_dir = "/content/data/11-785-s23-hw3p2" + "/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

!kaggle competitions submit -c automatic-speech-recognition-asr-slack-kaggle -f submission.csv -m "460"


100% 209k/209k [00:00<00:00, 430kB/s]
Successfully submitted to Automatic Speech Recognition (ASR) Slack Kaggle